In [5]:
# get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "asia-northeast1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [7]:
# define project information
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]"

# generate a unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")

In [8]:
BUCKET_URI = f"gs://{PROJECT_ID}-vs-quickstart-{UID}"

In [9]:
! gcloud storage buckets create $BUCKET_URI --location=$LOCATION --project=$PROJECT_ID
! gcloud storage cp "gs://github-repo/data/vs-quickstart/product-embs.json" $BUCKET_URI

Creating gs://gen-lang-client-0471694923-vs-quickstart-05130521/...
Copying gs://github-repo/data/vs-quickstart/product-embs.json to gs://gen-lang-client-0471694923-vs-quickstart-05130521/product-embs.json
  Completed files 1/1 | 79.3MiB/79.3MiB                                        


In [10]:
! gcloud storage cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests

Copying gs://github-repo/data/vs-quickstart/product-embs.json to file://./product-embs.json
  Completed files 1/1 | 79.3MiB/79.3MiB | 11.7MiB/s                            

Average throughput: 11.4MiB/s


In [11]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [12]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = f"vs-quickstart-index-{UID}",
    contents_delta_uri = BUCKET_URI,
    dimensions = 768,
    approximate_neighbors_count = 100,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/1010478563120/locations/asia-northeast1/indexes/2772229453437206528/operations/5909718318889041920
MatchingEngineIndex created. Resource name: projects/1010478563120/locations/asia-northeast1/indexes/2772229453437206528
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/1010478563120/locations/asia-northeast1/indexes/2772229453437206528')


In [13]:
## create `IndexEndpoint`
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = f"vs-quickstart-index-endpoint-{UID}",
    public_endpoint_enabled=False,
    enable_private_service_connect=True,
    project_allowlist=[PROJECT_ID],
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120/operations/8785829630918524928
MatchingEngineIndexEndpoint created. Resource name: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120')


In [14]:
DEPLOYED_INDEX_ID = f"vs_quickstart_deployed_{UID}"

In [15]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(
    index = my_index, deployed_index_id = DEPLOYED_INDEX_ID,
        psc_automation_configs=[("gen-lang-client-0471694923", "projects/1010478563120/global/networks/try-vpc")]
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120/operations/6205267044435230720
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120


resource name: projects/1010478563120/locations/asia-northeast1/indexEndpoints/1995745545808773120